# Setting up the Project.

In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

### Importing the libraries and importing the data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
df = pd.read_csv('/content/dataset.csv')

In [ ]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00


It is clear now that the data is sorted for all the 14 parking spaces, according to the timestamp of the sent data.

Going through the dataset we find out that, there are no missing values in the dataset, but there are categorical columns.

To engineer these data categories into useful data for our pricing model.

We'll use **OneHotEncoding**.


The columns we need to set up the encoding for are :
- Vehicle Type
- System Code Number




There is another column that we need to edit for easy access and calculation of data given to us.

- Lattitude
- Longitude

They are essentially signifying the location of the parking lot, for which we can set correlation of price with the nearby spots.

But before we begin, let's understand and stream the data accordingly. Since the first model only uses 4 ( 3 features + 1 Category ) columns.

In [ ]:
df[['Timestamp', 'Occupancy', 'Capacity', 'SystemCodeNumber']].to_csv("parking_stream.csv", index=False)

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    SystemCodeNumber: str  # Unique identifier for the parking location

In [ ]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [ ]:
print(data.schema)

id          | Timestamp | Occupancy | Capacity | SystemCodeNumber
ANY_POINTER | STR       | INT       | INT      | STR             


In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance= (pw.this.day, pw.this.SystemCodeNumber), # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        day=pw.this._pw_instance[0],                     # extract 'day' from instance
        SystemCodeNumber=pw.this._pw_instance[1],        # extract parking lot ID
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)



In [ ]:
pn.extension()
from bokeh.palettes import Category20
import bokeh.plotting

In [ ]:
def multi_lot_price_plotter(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Real-Time Parking Prices for All Lots",
        x_axis_type="datetime",
    )
    lots = list(set(source.data['SystemCodeNumber']))
    palette = Category20[20]  # 20 distinct colors
    for i, lot in enumerate(lots):
        mask = [x == lot for x in source.data['SystemCodeNumber']]
        fig.line(
            [x for x, m in zip(source.data['t'], mask) if m],
            [y for y, m in zip(source.data['price'], mask) if m],
            color=palette[i % 20],
            legend_label=str(lot),
            line_width=2,
        )
    fig.legend.title = "Parking Lot"
    fig.legend.click_policy = "hide"
    fig.xaxis.axis_label = "Timestamp"
    fig.yaxis.axis_label = "Price"
    return fig

In [ ]:
# viz = delta_window.plot(multi_lot_price_plotter, sorting_col="t")
# pn.Column(viz)

/tmp/ipython-input-14-529613173.py:19: UserWarning: 
You are attempting to set `plot.legend.title` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  fig.legend.title = "Parking Lot"
/tmp/ipython-input-14-529613173.py:20: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  fig.legend.click_policy = "hide"


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.io.jsonlines.write(delta_window, "parking_prices.jsonlines")
#pw.run()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [ ]:
# import json
# import pandas as pd

# # Read all rows from the JSONLines file into a list of dicts
# data = []
# with open("parking_prices.jsonlines", "r") as f:
#     for line in f:
#         try:
#             row = json.loads(line)
#             data.append(row)
#         except json.JSONDecodeError:
#             continue  # Skip bad lines

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Convert the timestamp column to datetime
# df["t"] = pd.to_datetime(df["t"])


In [ ]:
# # Set plot size
# plt.figure(figsize=(12, 6))

# # Group by SystemCodeNumber (i.e., parking lots)
# grouped = df.groupby("SystemCodeNumber")

# # Plot each lot in a separate subplot
# num_lots = len(grouped)
# fig, axes = plt.subplots(num_lots, 1, figsize=(12, 4 * num_lots), sharex=True)

# # If only one lot, axes is not a list
# if num_lots == 1:
#     axes = [axes]
# colours = [
#     'blue',
#     'orange',
#     'green',
#     'red',
#     'purple',
#     'brown',
#     'pink',
#     'gray',
#     'olive',
#     'cyan',
#     'gold',
#     'darkturquoise',
#     'mediumvioletred',
#     'limegreen'
# ]
# for ax, (lot, group), color in zip(axes, grouped, colours):
#     ax.plot(group["t"], group["price"], label=lot, color=color, marker="o")
#     ax.set_title(f"Price Plot for Lot: {lot}")
#     ax.set_ylabel("Price")
#     ax.grid(True)

# plt.xlabel("Timestamp")
# plt.tight_layout()
# plt.show()

In [ ]:
df = pd.read_csv('/content/dataset.csv')

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
print(df["TrafficConditionNearby"].unique())
print(df["VehicleType"].unique())

['low' 'average' 'high']
['car' 'bike' 'cycle' 'truck']


In [ ]:
traffic_weights = {
    "low": 0.2,
    "medium": 0.5,
    "high": 0.8
}

In [ ]:
df["TrafficWeight"] = df["TrafficConditionNearby"].map(traffic_weights)

In [ ]:
vehicle_weights = {
    "car": 1.0,
    "bike": 0.5,
    "cycle": 0.2,
    "truck": 1.5
}


df["VehicleTypeWeight"] = df["VehicleType"].map(vehicle_weights)

In [ ]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [ ]:
# Replace '.' or empty strings with NaN, then drop or fill
df.replace('.', np.nan, inplace=True)
df.replace('', np.nan, inplace=True)

# Option 1: Drop rows with any NaNs in float fields
df.dropna(subset=["TrafficWeight", "VehicleTypeWeight", "OccupancyRate", "QueueLength"], inplace=True)

# Option 2: Fill them with 0 or a default
# df.fillna({"TrafficWeight": 0.0, "VehicleTypeWeight": 0.0, "OccupancyRate": 0.0, "QueueLength": 0.0}, inplace=True)


In [ ]:
float_columns = ['TrafficWeight', 'VehicleTypeWeight', 'OccupancyRate', 'QueueLength']
df[float_columns] = df[float_columns].astype(float)

In [ ]:
print(df[df['TrafficWeight'] == '.'])

Empty DataFrame
Columns: [ID, SystemCodeNumber, Capacity, Latitude, Longitude, Occupancy, VehicleType, TrafficConditionNearby, QueueLength, IsSpecialDay, LastUpdatedDate, LastUpdatedTime, Timestamp, TrafficWeight, VehicleTypeWeight, OccupancyRate]
Index: []


In [ ]:
print(df[df['OccupancyRate'] == '.'])

Empty DataFrame
Columns: [ID, SystemCodeNumber, Capacity, Latitude, Longitude, Occupancy, VehicleType, TrafficConditionNearby, QueueLength, IsSpecialDay, LastUpdatedDate, LastUpdatedTime, Timestamp, TrafficWeight, VehicleTypeWeight, OccupancyRate]
Index: []


In [ ]:
print(df[df['QueueLength'] == '.'])

Empty DataFrame
Columns: [ID, SystemCodeNumber, Capacity, Latitude, Longitude, Occupancy, VehicleType, TrafficConditionNearby, QueueLength, IsSpecialDay, LastUpdatedDate, LastUpdatedTime, Timestamp, TrafficWeight, VehicleTypeWeight, OccupancyRate]
Index: []


In [ ]:
print(df[df['TrafficWeight'] == '.'])

Empty DataFrame
Columns: [ID, SystemCodeNumber, Capacity, Latitude, Longitude, Occupancy, VehicleType, TrafficConditionNearby, QueueLength, IsSpecialDay, LastUpdatedDate, LastUpdatedTime, Timestamp, TrafficWeight, VehicleTypeWeight, OccupancyRate]
Index: []


In [ ]:
columns=['Timestamp', 'Occupancy', 'Capacity', 'SystemCodeNumber', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType', 'OccupancyRate', 'TrafficWeight', 'VehicleTypeWeight']
df[columns].to_csv("parking_stream_v2.csv", index=False)

In [ ]:
df = pd.read_csv("/content/parking_stream_v2.csv")
for col in df.select_dtypes(include='object').columns:
    print(f"Unique values in '{col}': {df[col].unique()}")

Unique values in 'Timestamp': ['2016-10-04 07:59:00' '2016-10-04 08:25:00' '2016-10-04 08:59:00' ...
 '2016-12-19 15:29:00' '2016-12-19 16:03:00' '2016-12-19 16:30:00']
Unique values in 'SystemCodeNumber': ['BHMBCCMKT01' 'BHMNCPHST01' 'BHMMBMMBX01' 'BHMNCPNST01' 'Shopping'
 'BHMEURBRD01' 'Broad Street' 'Others-CCCPS105a' 'Others-CCCPS119a'
 'BHMBCCTHL01' 'Others-CCCPS135a' 'Others-CCCPS202' 'Others-CCCPS8'
 'Others-CCCPS98']
Unique values in 'TrafficConditionNearby': ['low' 'high']
Unique values in 'VehicleType': ['car' 'bike' 'cycle' 'truck']


In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    OccupancyRate: float
    QueueLength: float
    TrafficWeight: float
    IsSpecialDay: int
    VehicleTypeWeight: float

In [ ]:
data = pw.demo.replay_csv("/content/parking_stream_v2.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
print(data_with_time.schema)

id          | Timestamp | SystemCodeNumber | Occupancy | Capacity | OccupancyRate | QueueLength | TrafficWeight | IsSpecialDay | VehicleTypeWeight | t               | day
ANY_POINTER | STR       | STR              | INT       | INT      | FLOAT         | FLOAT       | FLOAT         | INT          | FLOAT             | DATE_TIME_NAIVE | STR


In [ ]:
import datetime

windowed = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.SystemCodeNumber),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        day = pw.this._pw_instance[0],
        SystemCodeNumber = pw.this._pw_instance[1],
        capacity = pw.reducers.any(pw.this.Capacity),
        occ_sum = pw.reducers.sum(pw.this.OccupancyRate),
        occ_count = pw.reducers.count(pw.this.OccupancyRate),
        qlen_sum = pw.reducers.sum(pw.this.QueueLength),
        qlen_count = pw.reducers.count(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.TrafficWeight),
        traffic_count = pw.reducers.count(pw.this.TrafficWeight),
        special_day = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight_sum = pw.reducers.sum(pw.this.VehicleTypeWeight),
        vehicle_weight_count = pw.reducers.count(pw.this.VehicleTypeWeight)
    )
    .select(
        pw.this.t,
        pw.this.day,
        pw.this.SystemCodeNumber,
        occ_avg = pw.this.occ_sum / pw.this.occ_count,
        qlen_avg = pw.this.qlen_sum / pw.this.qlen_count,
        traffic_avg = pw.this.traffic_sum / pw.this.traffic_count,
        special_day = pw.this.special_day,
        vehicle_weight_avg = pw.this.vehicle_weight_sum / pw.this.vehicle_weight_count,
        capacity = pw.this.capacity,

    )
)


In [ ]:
print(windowed.schema)


id                                                                                                                   | t               | day | SystemCodeNumber | occ_avg | qlen_avg | traffic_avg | special_day | vehicle_weight_avg | capacity
Pointer(Tuple(Tuple(STR, STR), DATE_TIME_NAIVE, DATE_TIME_NAIVE), DATE_TIME_NAIVE, DATE_TIME_NAIVE, Tuple(STR, STR)) | DATE_TIME_NAIVE | STR | STR              | FLOAT   | FLOAT    | FLOAT       | INT         | FLOAT              | INT     


In [ ]:
alpha = 1.0
beta = 0.6
gamma = 0.8
delta = 0.5
epsilon = 0.7
base_price = 10
lambda_ = 1.0

In [ ]:
# Step 1: Add demand column
with_demand = windowed.with_columns(
    demand=(
        alpha * pw.this.occ_avg / pw.this.capacity
        + beta * pw.this.qlen_avg
        - gamma * pw.this.traffic_avg
        + delta * pw.this.special_day
        + epsilon * pw.this.vehicle_weight_avg
    )
)


pw.io.jsonlines.write(with_demand, "debug_demand.jsonlines")
#pw.run()

In [ ]:
# import pandas as pd
# import json

# # Read the JSONLines file into a list of dicts
# data = []
# with open("/content/debug_demand.jsonlines", "r") as f:
#     for line in f:
#         try:
#             row = json.loads(line)
#             data.append(row)
#         except json.JSONDecodeError:
#             continue  # Skip bad lines

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Show min and max of the 'demand' column
# print("Min demand:", df["demand"].min())
# print("Max demand:", df["demand"].max())

In [ ]:
DEMAND_MIN = 1.9
DEMAND_MAX = 5.6




# Step 2: Add normalized_demand
with_normalized = with_demand.with_columns(
    normalized_demand=pw.apply_with_type(
        lambda d: max(0.0, min(1.0, (d - DEMAND_MIN) / (DEMAND_MAX - DEMAND_MIN))),
        float,
        pw.this.demand
    )
)


pw.io.jsonlines.write(with_normalized, "debug_normalized.jsonlines")
# pw.run()

In [ ]:

# Step 3: Add price_raw
with_price_raw = with_normalized.with_columns(
    price_raw=10 * (1 + lambda_ * pw.this.normalized_demand)
)

# Step 4: Clamp final price
final = with_price_raw.with_columns(
    price=pw.apply_with_type(
        lambda p: max(base_price * 0.5, min(base_price * 2.0, p)),
        float,
        pw.this.price_raw
    )
)


In [ ]:
pn.extension()
viz = delta_window.plot(multi_lot_price_plotter, sorting_col="t")
viz2 = final.plot(multi_lot_price_plotter, sorting_col="t")
pn.Column(viz, viz2)

/tmp/ipython-input-14-529613173.py:19: UserWarning: 
You are attempting to set `plot.legend.title` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  fig.legend.title = "Parking Lot"
/tmp/ipython-input-14-529613173.py:20: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  fig.legend.click_policy = "hide"


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.io.jsonlines.write(final, "demand_based_pricing.jsonlines")
# pw.run()

In [ ]:
# import json
# import pandas as pd

# # Read all rows from the JSONLines file into a list of dicts
# data = []
# with open("/content/demand_based_pricing.jsonlines", "r") as f:
#     for line in f:
#         try:
#             row = json.loads(line)
#             data.append(row)
#         except json.JSONDecodeError:
#             continue  # Skip bad lines

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Convert the timestamp column to datetime
# df["t"] = pd.to_datetime(df["t"])

In [ ]:
# # Set plot size
# plt.figure(figsize=(12, 6))

# # Group by SystemCodeNumber (i.e., parking lots)
# grouped = df.groupby("SystemCodeNumber")

# # Plot each lot in a separate subplot
# num_lots = len(grouped)
# fig, axes = plt.subplots(num_lots, 1, figsize=(12, 4 * num_lots), sharex=True)

# # If only one lot, axes is not a list
# if num_lots == 1:
#     axes = [axes]
# colours = [
#     'blue',
#     'orange',
#     'green',
#     'red',
#     'purple',
#     'brown',
#     'pink',
#     'gray',
#     'olive',
#     'cyan',
#     'gold',
#     'darkturquoise',
#     'mediumvioletred',
#     'limegreen'
# ]
# for ax, (lot, group), color in zip(axes, grouped, colours):
#     ax.plot(group["t"], group["price"], label=lot, color=color, marker="o")
#     ax.set_title(f"Price Plot for Lot: {lot}")
#     ax.set_ylabel("Price")
#     ax.grid(True)

# plt.xlabel("Timestamp")
# plt.tight_layout()
# plt.show()

In [ ]:
pw.run()

Output()